
# Voorbereiding en controle-rapportages rondom proces jaarlijkse huuraanpassing!

- [Specificatie per 30-6 en 1-7](#100)
- [Eenheden met brutohuurprijs onder de 1 euro](#110)
- [Check op bepaalde elementen die niet meer voor mogen komen](#120)
- [Eenheden met 061 Huurkorting per 30-06 en 1-7 van lopend jaar](#130)
- [Eenheden met 413 Woonduurkorting 10 jaar - afwijkend tov -0.97](#140)
- [480 Huurcorrectie eigen gebruik + 481 Servicekostencorrectie eigen gebruik](#150)
- [008 Standplaats + 482 Huurcorrectie ivm extern beheer](#160)
- [Overzicht alle elementen per 30-6, volgens te muteren tabel, per 1-7](#170)
- [Check nieuwe netto huur simulatie met functie netto huur op basis van kopie element\contract-tabel](#180)
- [Check specificatie huur per eenheid is optelsom van huur per element](#190)
- [BIJLAGE 1: rapportage-onderdelen tbv genereren huuraanpassing (maatwerk Jaco/Eric Reitsma in Empire)]
- [BIJLAGE 2: welke tabellen worden gekopieerd en bewaard]
- [BIJLAGE 3: welke database-objecten zijn in gebruik]

<a id=00 /> Jaco van der Wel, 14 april 2021

- NB: alles op basis van bevroren huurverhogings-regels. Dat wil zeggen gekopieerde/bewaarde tabellen op staedion\_dm.huuraanpassing
- Zodoende kun je ook de werkelijke 1-7 regels vergelijken met de bevroren regels

<a id=100 />
## Specificatie per 30-6 en 1-7
- staedion\_dm.Huuraanpassing.Specificatie\_30\_juni
- staedion\_dm.Huuraanpassing.Specificatie\_30\_juni
-

In [ ]:
use staedion_dm
;

Drop table If Exists Huuraanpassing.Specificatie_30_juni
;
SELECT  Peildatum = datefromparts(year(getdate()),6,30)
	, Eenheidnr
	, Volgnummer
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
	, ElementNaam = Elementnr + ' ' + [Omschrijving element]
    , [Omschrijving element]
    , Bedrag = Bedrag --convert(DECIMAL(12, 2), Bedrag)
	, [Bedrag niet afgerond] 
	, [Bedrag incl. BTW]
	, [BTW-productboekingsgroep]
    , [Status ogekaart]
	, [Op huurverhoging afdrukken]
    , [Eenmalig]    
into staedion_dm.Huuraanpassing.Specificatie_30_juni
FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),6,30)) AS BASIS
ORDER BY Eenheidnr
      ,[Elementnr]

Drop table If Exists Huuraanpassing.Specificatie_1_juli
;
SELECT  Peildatum = datefromparts(year(getdate()),7,1)
	, Eenheidnr
	, Volgnummer
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
	, ElementNaam = Elementnr + ' ' + [Omschrijving element]
    , [Omschrijving element]
    , Bedrag = Bedrag --convert(DECIMAL(12, 2), Bedrag)
	, [Bedrag niet afgerond] 
	, [Bedrag incl. BTW]
	, [BTW-productboekingsgroep]
    , [Status ogekaart]
	, [Op huurverhoging afdrukken]
    , [Eenmalig]
into staedion_dm.Huuraanpassing.Specificatie_1_juli
FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),7,1)) AS BASIS
ORDER BY Eenheidnr
      ,[Elementnr]      

Drop table If Exists Huuraanpassing.Huurprijs_1_juli
;
SELECT Eenheidnr = OGE.Nr_
	   ,Adres = OGE.Straatnaam + ' ' + OGE.Huisnr_  + ' ' + OGE.Toevoegsel
       ,CLUS.Clusternr
       ,CLUS.Clusternaam
       ,CONT.Assetmanager
       ,Peildatum = getdate()
       ,Huurbeleid = CASE oge.huurbeleid
              WHEN 0
                     THEN 'Hoogste bedrag'
              WHEN 1
                     THEN 'Streefhuur'
              WHEN 2
                     THEN 'Elementsjabloon'
              WHEN 3
                     THEN 'Huidige huur'
              WHEN 4
                     THEN 'Markthuurwaarde'
              END
       ,[Nettohuur] = convert(FLOAT, HPR.nettohuur)
       ,[Nettohuur incl btw] = convert(FLOAT, HPR.nettohuur_inclbtw)
       ,[Brutohuur] = convert(FLOAT, HPR.brutohuur_inclbtw)
       ,[Brutohuur incl btw] = convert(FLOAT, HPR.brutohuur_inclbtw)
       ,HPR.prolongatietermijn
       ,KENM.[Corpodata type]
into staedion_dm.Huuraanpassing.Huurprijs_1_juli
FROM empire_data.dbo.Staedion$oge AS OGE
OUTER APPLY staedion_dm.Huuraanpassing.fn_Huurprijs(OGE.Nr_, datefromparts(year(getdate()),7,1)) AS HPR
OUTER APPLY staedion_dm.Eenheden.fnEigenschappen(OGE.Nr_, getdate()) AS KENM
OUTER APPLY empire_staedion_data.[dbo].ITVfnCLusterBouwblok(OGE.Nr_) AS CLUS
OUTER APPLY empire_staedion_data.[dbo].[ITVFnContactbeheer](OGE.Nr_) AS CONT
--OUTER APPLY empire_staedion_data.[dbo].ITVfnContractaanhef(HPR.huurdernr) AS HRD
WHERE OGE.[Common Area] = 0
       AND OGE.[Begin exploitatie] <> '17530101'
       AND OGE.[Begin exploitatie] <= datefromparts(year(getdate()),7,1)
       AND (
              OGE.[Einde exploitatie] = '17530101'
              OR OGE.[Einde exploitatie] > datefromparts(year(getdate()),7,1)
              );

select count(*) from staedion_dm.Huuraanpassing.Specificatie_30_juni;
select count(*) from staedion_dm.Huuraanpassing.Specificatie_1_juli;
select count(*) from staedion_dm.Huuraanpassing.Huurprijs_1_juli;


<a id=110 />

## Eenheden met brutohuurprijs onder de 1 euro
- [Terug naar inhoudsopgave](#00)

In [ ]:
SELECT Eenheidnr = OGE.Nr_
	   ,Adres = OGE.Straatnaam + ' ' + OGE.Huisnr_  + ' ' + OGE.Toevoegsel
       ,CLUS.Clusternr
       ,CLUS.Clusternaam
       ,CONT.Assetmanager
       ,Peildatum = getdate()
       ,Huurbeleid = CASE oge.huurbeleid
              WHEN 0
                     THEN 'Hoogste bedrag'
              WHEN 1
                     THEN 'Streefhuur'
              WHEN 2
                     THEN 'Elementsjabloon'
              WHEN 3
                     THEN 'Huidige huur'
              WHEN 4
                     THEN 'Markthuurwaarde'
              END
       ,[Nettohuur] = convert(FLOAT, HPR.nettohuur)
       ,[Nettohuur incl btw] = convert(FLOAT, HPR.nettohuur_inclbtw)
       ,[Brutohuur] = convert(FLOAT, HPR.brutohuur_inclbtw)
       ,[Brutohuur incl btw] = convert(FLOAT, HPR.brutohuur_inclbtw)
       ,KENM.[Corpodata type]
--into staedion_dm.Huuraanpassing.Huurprijs_1_juli
FROM empire_data.dbo.Staedion$oge AS OGE
OUTER APPLY staedion_dm.Huuraanpassing.fn_Huurprijs(OGE.Nr_, datefromparts(year(getdate()),7,1)) AS HPR
OUTER APPLY staedion_dm.Eenheden.fnEigenschappen(OGE.Nr_, getdate()) AS KENM
OUTER APPLY empire_staedion_data.[dbo].ITVfnCLusterBouwblok(OGE.Nr_) AS CLUS
OUTER APPLY empire_staedion_data.[dbo].[ITVFnContactbeheer](OGE.Nr_) AS CONT
--OUTER APPLY empire_staedion_data.[dbo].ITVfnContractaanhef(HPR.huurdernr) AS HRD
WHERE OGE.[Common Area] = 0
       AND HPR.brutohuur_inclbtw <= 1 
       AND OGE.[Begin exploitatie] <> '17530101'
       AND OGE.[Begin exploitatie] <= datefromparts(year(getdate()),7,1)
       AND (
              OGE.[Einde exploitatie] = '17530101'
              OR OGE.[Einde exploitatie] > datefromparts(year(getdate()),7,1)
              );

[Check op bepaalde elementen die niet meer voor mogen komen- [Terug naar inhoudsopgave](#00)](..\..\..\..\..\..)

In [ ]:
/* Check op aanwezigheid van niet-actieve elementen */
DROP TABLE IF EXISTS #cte_InActieveElementen ;

WITH cte_InActieveElementen
AS (
	SELECT ElementNummer
		,SoortVerhogingAlgemeen = CASE ALG.SoortVerhogingAlgemeen
			WHEN 0
				THEN 'Niet'
			WHEN 1
				THEN '%'
			WHEN 2
				THEN 'Bedrag'
			END
		,ALG.VerhogingAlgemeen
		,ELEM.Productboekingsgroep
		,ELEM.Omschrijving
	FROM empire.empire.dbo.HuurverhogingAlgemeen AS ALG
	LEFT OUTER JOIN empire.empire.dbo.staedion$element  AS ELEM with (nolock) ON ELEM.Nr_ = ALG.elementnummer
		AND ELEM.TAbel = 0
	WHERE ALG.ElementNummer IN (
			'124'
			,'129'
			,'201'
			,'211'
			,'215'
			,'218'
			,'219'
			)
	)
SELECT * INTO #cte_InActieveElementen FROM cte_InActieveElementen
;
SELECT	ELEM.ElementNummer,  ELEM.Omschrijving, [Aantal regels per 1-7] = count(*)
FROM	#cte_InActieveElementen as ELEM
cross apply staedion_dm. Huuraanpassing.[fn_ElementenDetails] (null,'20210701') as DET
group by ELEM.ElementNummer,  ELEM.Omschrijving
order by ELEM.ElementNummer,  ELEM.Omschrijving
;


[Eenheden met 061 Huurkorting per 30-06 en 1-7 van lopend jaar (op basis van bevroren huurverhogings-regels)- [Terug naar inhoudsopgave](#00)](..\..\..\..\..\..)

In [ ]:
SELECT Peildatum = datefromparts(year(getdate()),6,30)
	, Eenheidnr
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = Bedrag--convert(DECIMAL(12, 2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),6,30)) AS BASIS
WHERE EXISTS (
        SELECT 1
        FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),6,30)) AS KOPIE
        WHERE KOPIE.Elementnr IN ('061')
            AND BASIS.Eenheidnr = KOPIE.Eenheidnr
        )
UNION
SELECT Peildatum = datefromparts(year(getdate()),7,1)
	, Eenheidnr
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = Bedrag --convert(DECIMAL(12, 2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),7,1)) AS BASIS
WHERE EXISTS (
        SELECT 1
        FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),7,1)) AS KOPIE
        WHERE KOPIE.Elementnr IN ('061')
            AND BASIS.Eenheidnr = KOPIE.Eenheidnr
        )
ORDER BY Eenheidnr
      ,Peildatum
    , [Elementnr];


<a id=140 />

## Eenheden met 413 Woonduurkorting 10 jaar - afwijkend tov -0.97
- [Terug naar inhoudsopgave](#00)

In [ ]:
SELECT Peildatum = datefromparts(year(getdate()),7,1)
	, Eenheidnr
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel]= convert(nvarchar(20),[Einddatum contractregel],105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = convert(decimal(12,2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),7,1))
WHERE Elementnr IN ('413')
    AND Bedrag <> - 0.97
UNION
SELECT Peildatum = datefromparts(year(getdate()),6,30)
	, Eenheidnr
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel]= convert(nvarchar(20),[Einddatum contractregel],105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = convert(decimal(12,2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[fn_ElementenDetails](NULL, datefromparts(year(getdate()),6,30))
WHERE Elementnr IN ('413')
    AND Bedrag <> - 0.97
ORDER BY Eenheidnr
    , Peildatum
    , [Elementnr];


[480 Huurcorrectie eigen gebruik + 481 Servicekostencorrectie eigen gebruik- [Terug naar inhoudsopgave](#00)](..\..\..\..\..\..)

In [ ]:
SELECT Peildatum = datefromparts(year(getdate()),6,30)
	, Eenheidnr
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = convert(DECIMAL(12, 2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),6,30)) AS BASIS
WHERE EXISTS (
        SELECT 1
        FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),6,30)) AS KOPIE
        WHERE KOPIE.Elementnr IN ('480','481')
            AND BASIS.Eenheidnr = KOPIE.Eenheidnr
        )
union

SELECT Peildatum = datefromparts(year(getdate()),7,1)
	, Eenheidnr
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = convert(DECIMAL(12, 2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),7,1)) AS BASIS
WHERE EXISTS (
        SELECT 1
        FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),7,1)) AS KOPIE
        WHERE KOPIE.Elementnr IN ('480','481')
            AND BASIS.Eenheidnr = KOPIE.Eenheidnr
        )

ORDER BY Eenheidnr
      ,Peildatum
    , [Elementnr];


[008 Standplaats + 482 Huurcorrectie ivm extern beheer- [Terug naar inhoudsopgave](#00)](..\..\..\..\..\..)

In [ ]:
SELECT Peildatum = datefromparts(year(getdate()),6,30)
	, Eenheidnr
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = convert(DECIMAL(12, 2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),6,30)) AS BASIS
WHERE EXISTS (
        SELECT 1
        FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),6,30)) AS KOPIE
        WHERE KOPIE.Elementnr IN ('008','482')
            AND BASIS.Eenheidnr = KOPIE.Eenheidnr
        )
union

SELECT Peildatum = datefromparts(year(getdate()),7,1)
	, Eenheidnr
    /*, [Ingangsdatum contractregel] = convert(NVARCHAR(20), [Ingangsdatum contractregel], 105)  */
    , [Ingangsdatum contractregel]= convert(nvarchar(20),[Ingangsdatum contractregel],105)
    , [Einddatum contractregel] =  convert(NVARCHAR(20), [Einddatum contractregel], 105)
    , [Huurdernaam]
    , [Elementnr]
    , [Omschrijving element]
    , Bedrag = convert(DECIMAL(12, 2), Bedrag)
    , [Status ogekaart]
FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),7,1)) AS BASIS
WHERE EXISTS (
        SELECT 1
        FROM staedion_dm.Huuraanpassing.[ElementenDetails](NULL, datefromparts(year(getdate()),7,1)) AS KOPIE
        WHERE KOPIE.Elementnr IN ('008','482')
            AND BASIS.Eenheidnr = KOPIE.Eenheidnr
        )

ORDER BY Eenheidnr
      ,Peildatum
    , [Elementnr];

[Overzicht alle elementen per 30-6, volgens te muteren tabel, per 1-7- [Terug naar inhoudsopgave](#00)](..\..\..\..\..\..)

In [ ]:
DROP TABLE IF EXISTS #cte_InActieveElementen ;
DROP TABLE IF EXISTS #cte_TrendMatigeElementen ;
DROP TABLE IF EXISTS #cte_17 ;
DROP TABLE IF EXISTS #cte_306 ;
DROP TABLE IF EXISTS #cte_sim ;

WITH cte_InActieveElementen
AS (
	SELECT ElementNummer
		,SoortVerhogingAlgemeen = CASE ALG.SoortVerhogingAlgemeen
			WHEN 0
				THEN 'Niet'
			WHEN 1
				THEN '%'
			WHEN 2
				THEN 'Bedrag'
			END
		,ALG.VerhogingAlgemeen
		,ELEM.Productboekingsgroep
		,ELEM.Omschrijving
	FROM empire.empire.dbo.HuurverhogingAlgemeen AS ALG
	LEFT OUTER JOIN empire.empire.dbo.staedion$element AS ELEM ON ELEM.Nr_ = ALG.elementnummer
		AND ELEM.TAbel = 0
	WHERE ALG.ElementNummer IN (
			'124'
			,'129'
			,'201'
			,'211'
			,'215'
			,'218'
			,'219'
			)
	)
SELECT * INTO #cte_InActieveElementen FROM cte_InActieveElementen
;

WITH cte_TrendMatigeElementen
AS (
	SELECT ElementNummer
		,SoortVerhogingAlgemeen = CASE ALG.SoortVerhogingAlgemeen
			WHEN 0
				THEN 'Niet'
			WHEN 1
				THEN '%'
			WHEN 2
				THEN 'Bedrag'
			END
		,ALG.VerhogingAlgemeen
		,ELEM.Productboekingsgroep
		,ELEM.Omschrijving
	FROM empire.empire.dbo.HuurverhogingAlgemeen AS ALG
	LEFT OUTER JOIN empire.empire.dbo.staedion$element AS ELEM ON ELEM.Nr_ = ALG.elementnummer
		AND ELEM.TAbel = 0
	WHERE ALG.ElementNummer IN (
			'101'
			,'103'
			,'107'
			,'109'
			,'111'
			,'113'
			,'117'
			,'121'
			,'123'
			,'125'
			,'126'
			,'127'
			,'131'
			,'133'
			,'137'
			,'138'
			,'141'
			,'145'
			,'148'
			,'151'
			,'161'
			,'163'
			,'166'
			,'206'
			,'207'
			,'208'
			,'212'
			,'213'
			,'224'
			,'225'
			,'352'
			)
	)
SELECT * INTO #cte_TrendMatigeElementen FROM cte_TrendMatigeElementen
;
;WITH cte_17 (
	[SVKN-cluster]
	,Elementnr
	,[Aantal eenheden 1-7]
	,[Totaal bedrag 1-7]
	,[Gemiddeld bedrag 1-7]
	)
AS (
	SELECT icb.[SVKN-cluster]
		,ELEM.Elementnr
		,[Aantal eenheden 1-7] = COUNT(DISTINCT Eenheidnr)
		,[Totaal bedrag 1-7] = sum(Bedrag)
		,[Gemiddeld bedrag 1-7] = AVG(Bedrag)
	FROM empire_staedion_data.hvh.[ElementenDetails](NULL, '20210701') AS ELEM
	OUTER APPLY empire_staedion_data.dbo.ITVfnCLusterBouwblok(ELEM.Eenheidnr) AS icb
	GROUP BY ELEM.Elementnr
		,icb.[SVKN-cluster]
	)
SELECT * INTO #cte_17 FROM cte_17
;
WITH cte_306 (
	[SVKN-cluster]
	,Elementnr
	,[Aantal eenheden 30-6]
	,[Totaal bedrag 30-6]
	,[Gemiddeld bedrag 30-6]
	)
AS (
	SELECT icb.[SVKN-cluster]
		,ELEM.Elementnr
		,[Aantal eenheden 30-6] = COUNT(DISTINCT Eenheidnr)
		,[Totaal bedrag 30-6] = sum(Bedrag)
		,[Gemiddeld bedrag 30-6] = AVG(Bedrag)
	FROM empire_staedion_data.hvh.[ElementenDetails](NULL, '20210630') AS ELEM
	OUTER APPLY empire_staedion_data.dbo.ITVfnCLusterBouwblok(ELEM.Eenheidnr) AS icb
	GROUP BY ELEM.Elementnr
		,icb.[SVKN-cluster]
	)
SELECT * INTO #cte_306 FROM cte_306
;
WITH cte_sim (
	Eenheidnummer
	,Elementnr
	,[Te verwachten bedrag per 1-7]
	)
AS (SELECT HVH.Eenheidnummer
	,HVH.ElementNummer
	,[Te verwachten bedrag per 1-7] =  CASE HVH.SoortVerhogingEenheid
		WHEN 1
			THEN ROUND(ELEM.[Eenheidprijs (LV)]*1.00*(HVH.WaardeVerhogingEenheid+100)/100,2)
		WHEN 2
			THEN ROUND(ELEM.[Eenheidprijs (LV)]+ HVH.WaardeVerhogingEenheid,2)
		END
FROM empire.empire.dbo.HuurverhogingEenheid AS HVH
LEFT OUTER JOIN empire.empire.dbo.Staedion$Element AS ELEM 
	ON ELEM.Eenheidnr_ = HVH.Eenheidnummer
	AND ELEM.Volgnummer = HVH.Volgnummer
	AND ELEM.Nr_ = HVH.ElementNummer
	--WHERE ELEM.Nr_ BETWEEN '100' AND '299'
	)
SELECT * into #cte_sim FROM cte_sim

SELECT Elementnr = ELEM.Nr_
		,ELEM.Omschrijving
		,[Totaal bedrag 30-6] = (select SUM(_306.[Totaal bedrag 30-6]) from #cte_306 AS _306 where ELEM.Nr_ = _306.Elementnr)
		,[Te verwachten bedrag per 1-7] = convert(float,(select SUM(_s.[Te verwachten bedrag per 1-7]) from #cte_sim  _s where ELEM.Nr_ = _s.Elementnr ))
		,[Totaal bedrag 1-7] = convert(float,(select SUM(_17.[Totaal bedrag 1-7]) from #cte_17 AS _17 where ELEM.Nr_ = _17.Elementnr))
		,[Aantal eenheden 30-6] = convert(float,(select sum(distinct _17.[Aantal eenheden 30-6]) from #cte_306 AS _17 where ELEM.Nr_ = _17.Elementnr))
		,[Aantal eenheden 1-7] = convert(float,(select sum(distinct _17.[Aantal eenheden 1-7]) from #cte_17 AS _17 where ELEM.Nr_ = _17.Elementnr))
		,[Percentage werkelijke verandering]  = 
					iif(convert(float,(select SUM(_306.[Totaal bedrag 30-6]) from #cte_306 AS _306 where ELEM.Nr_ = _306.Elementnr)) <>0
							,round( (select SUM(_17.[Totaal bedrag 1-7]) from #cte_17 AS _17 where ELEM.Nr_ = _17.Elementnr)
								/ (select SUM(_306.[Totaal bedrag 30-6]) from #cte_306 AS _306 where ELEM.Nr_ = _306.Elementnr),4)
								,null)
		,Trendmatig = iif(coalesce(max(TM.ElementNummer), '')='','nvt','Trendmatig')
		,Inactief = iif(coalesce(max(IA.ElementNummer), '')='','nvt','Inactief')
		,ELEM.soort
FROM empire.empire.dbo.staedion$element AS ELEM
left outer join #cte_InActieveElementen as IA on IA.ElementNummer =ELEM.Nr_
left outer join #cte_TrendMatigeElementen as TM on TM.ElementNummer =ELEM.Nr_
where ELEM.TAbel = 0 and ELEM.soort = 0
GROUP BY ELEM.Nr_,ELEM.Omschrijving,ELEM.soort
HAVING convert(float,(select SUM(_306.[Totaal bedrag 30-6]) from #cte_306 AS _306 where ELEM.Nr_ = _306.Elementnr)) is not NULL
        or convert(float,(select SUM(_s.[Te verwachten bedrag per 1-7]) from #cte_sim  _s where ELEM.Nr_ = _s.Elementnr )) is not NULL
        or convert(float,(select SUM(_17.[Totaal bedrag 1-7]) from #cte_17 AS _17 where ELEM.Nr_ = _17.Elementnr)) is not null
order by ELEM.Nr_
;



## Check nieuwe netto huur simulatie met functie netto huur op basis van kopie element\contract-tabel
- [Terug naar inhoudsopgave](#00)

Dat dient 2 doelen: is simulatiebestand consistent met 1-7-regel + is functie ok](..\..\..\..\..\..)

In [ ]:
/*  Check of Nieuwe nettohuur van simulatie ook overeenkomt met functie huurprijs op basis van hvh-tabellen element\contract
    (in deze functie wordt gebruik gemaakt van empire_staedion_data.hvh.fnHuurprijs */
SELECT	Huursomtijdvakcode, Eenheidnr,[Nieuwe nettohuur simulatie], [Nieuwe nettohuur 1-7], Verwerkingsstatus
FROM	staedion_dm.Huuraanpassing.[fn_SimulatieHuursom](DEFAULT) 
WHERE   [Nieuwe nettohuur simulatie] <> [Nieuwe nettohuur 1-7]
order by Eenheidnr


[Check specificatie huur per eenheid is optelsom van huur per element- [Terug naar inhoudsopgave](#00)Zijn beide bestanden consistent ?](..\..\..\..\..\..)

In [ ]:
Drop table If Exists #SPEC;
Drop table If Exists #TOT;

select Eenheidnr, Bedrag = sum(coalesce([Bedrag incl. BTW],0)) 
into #SPEC
from staedion_dm.huuraanpassing.Specificatie_1_juli
where Eenmalig = 0
group by Eenheidnr
;
select Eenheidnr, Prolongatietermijn, Bedrag = sum(coalesce([Brutohuur incl btw],0)) 
into #TOT
from staedion_dm.huuraanpassing.Huurprijs_1_juli
group by Eenheidnr, Prolongatietermijn
;
select Eenheid = coalesce(S.eenheidnr,T.eenheidnr) , Prolongatietermijn, Specificatie = S.Bedrag, Totaal = T.Bedrag, [Afwijking] = abs( coalesce(S.Bedrag,0) - coalesce(T.Bedrag,0))
from #SPEC as S
full outer join #TOT as T
on S.eenheidnr = T.eenheidnr
where  abs( coalesce(S.Bedrag,0) - coalesce(T.Bedrag,0))>=0.1
;

<span style="color: #800000;font-weight: bold;">##&nbsp;Check wijzigingen elementen</span>

  

\- \[<span style="color: #a31515;">Terug&nbsp;naar&nbsp;inhoudsopgave</span>\](<u>#00</u>)

  

Een overall check op korte termijn op onderstaande constatering is van belang!

Het lijkt erop dat (bij een nog onbekend aantal adressen) het verwijderen van element 206 niet goed is gegaan.

Zo te zien zijn er correcties nodig (en wellicht al communicatie hierover; naar KCC en/of specifieke klanten).

In [ ]:
-- Element 206 verwijderd in 2021 maar daar ging iets niet goed
select Datum = '20210701', Huurdernr, Eenheidnr, elementnr, [Omschrijving element], bedrag= convert(float,bedrag)
from staedion_dm.huuraanpassing.fn_ElementenDetails (null,'20210701') 
where elementnr in ('207', '206')
;
select Datum = '20210630',Huurdernr, Eenheidnr, elementnr,[Omschrijving element], bedrag= convert(float,bedrag)
from staedion_dm.huuraanpassing.fn_ElementenDetails (null,'20210630') 
where elementnr in ('207', '206')
;

# 1000

## BIJLAGE 1: rapportage-onderdelen tbv genereren huuraanpassing (maatwerk Jaco/Eric Reitsma in Empire)

### 1\. Importeren huurverhoging (rapport in rapportageportal + bijbehorende stored procedure in Empire-database)

<table class="ms-listviewtable" id="onetidDoclibViewTbl0" dir="none" border="0" cellspacing="0" cellpadding="1" summary="RapportenBoek" o:webquerysourcehref="&amp;XMLDATA=1&amp;RowLimit=0&amp;View=%7B082E8BDB%2D7144%2D41B0%2D8CD5%2DCE4F09CB41B0%7D" xmlns:o="urn:schemas-microsoft-com:office:office" handledeleteinit="true"><tbody id="tbod1-6__" isloaded="true" selectablerows="8"><tr class="ms-alternating  ms-itmHoverEnabled ms-itmhover s4-itm-selected" id="1,1543,0" iid="1,1543,0" setedgeborder="true"><td height="100%" class="ms-cellstyle ms-vb-title" iscallout="TRUE" isecb="TRUE"><div class="ms-vb  itx" id="1543" app="" ctxname="ctx1"><a class="ms-listlink ms-draggable" aria-label="Importeren huurverhoging PRODUCTIE, rdl Bestand" href="http://s-dwh2012-sp/Rapportenboek/Importeren%20huurverhoging%20PRODUCTIE.rdl" dragid="3">Importeren huurverhoging PRODUCTIE</a></div></td><td class="ms-list-itemLink-td ms-cellstyle"><div class="ms-list-itemLink  "><a title="Menu Openen" class="ms-lstItmLinkAnchor ms-ellipsis-a" id="ctx1_1543_calloutLaunchPoint" href="http://s-dwh2012-sp/Level%203%20Financin/Directie%20Financien.aspx#" ms-jsgrid-click-passthrough="true"><img class="ms-ellipsis-icon" alt="Importeren huurverhoging PRODUCTIE.rdl - menu Openen" src="http://s-dwh2012-sp/_catalogs/theme/Themed/7FA80357/spcommon-B35BB0A9.themedpng?ctag=2"></a></div></td><td class="ms-cellstyle ms-vb2 ms-vb-lastCell"><div class="ms-rtestate-field" dir="">Dit rapport controleert de gegevens die vanuit een excel-bestand zijn ingelezen in de tabel "empire_staedion_data.hvh.ImportElementHvH" en verwerkt deze in de maatwerk-tabel HuurverhogingEenheid<br><br>DRAAIBOEK: stap "Rapport vullen tabellen"<br>02-04-2020 Gekoppeld aan ingelezen lijst empire_staedion_data.hvh.ImportElementHvH): H:\Algemeen\2020 huurverhoging\Empire\Servicekosten\Afwijkingen per eenheid importbestand - versie Jaco, aanpassing Petra nr 6.xlsx; 03-04-2020 06:14</div></td></tr></tbody></table>

### <span style="color: #09885a;">2</span>. Afwijkingen op <span style="color: #0000ff;">service</span><span style="color: #000000;">-</span>clusterniveau <span style="color: #0000ff;">in</span> hulptabellen verwerken (rapport <span style="color: #0000ff;">in</span> rapportageportal <span style="color: #000000;">+</span> bijbehorende stored <span style="color: #0000ff;">procedure</span> <span style="color: #0000ff;">in</span> Empire<span style="color: #000000;">-</span><span style="color: #0000ff;">database</span>)

<table class="ms-listviewtable" id="onetidDoclibViewTbl0" dir="none" border="0" cellspacing="0" cellpadding="1" summary="RapportenBoek" o:webquerysourcehref="&amp;XMLDATA=1&amp;RowLimit=0&amp;View=%7B082E8BDB%2D7144%2D41B0%2D8CD5%2DCE4F09CB41B0%7D" xmlns:o="urn:schemas-microsoft-com:office:office" handledeleteinit="true"><tbody id="tbod1-6__" isloaded="true" selectablerows="8"><tr class=" ms-itmHoverEnabled ms-itmhover s4-itm-selected" id="1,1542,0" iid="1,1542,0" setedgeborder="true"><td height="100%" class="ms-cellstyle ms-vb-title" iscallout="TRUE" isecb="TRUE"><div class="ms-vb  itx" id="1542" app="" ctxname="ctx1"><a class="ms-listlink ms-draggable" aria-label="Vullen hulptabellen Huurverhoging cluster en eenheden - PRODUCTIE, rdl Bestand" href="http://s-dwh2012-sp/Rapportenboek/Vullen%20hulptabellen%20Huurverhoging%20cluster%20en%20eenheden%20-%20PRODUCTIE.rdl" dragid="9">Vullen hulptabellen Huurverhoging cluster en eenheden - PRODUCTIE</a></div></td><td class="ms-list-itemLink-td ms-cellstyle"><div class="ms-list-itemLink  "><a title="Menu Openen" class="ms-lstItmLinkAnchor ms-ellipsis-a" id="ctx1_1542_calloutLaunchPoint" href="http://s-dwh2012-sp/Level%203%20Financin/Directie%20Financien.aspx#" ms-jsgrid-click-passthrough="true"><img class="ms-ellipsis-icon" alt="Vullen hulptabellen Huurverhoging cluster en eenheden - PRODUCTIE.rdl - menu Openen" src="http://s-dwh2012-sp/_catalogs/theme/Themed/7FA80357/spcommon-B35BB0A9.themedpng?ctag=2"></a></div></td><td class="ms-cellstyle ms-vb2 ms-vb-lastCell"><div class="ms-rtestate-field" dir="">Dit rapport wist de tabellen HuurverhogingEenheid en HuurverhogingCluster en vult ze daarna weer. Deze tabellen worden gevuld op basis van: * de verhoging zoals die per element is vastgelegd in de tabel VerhogingAlgemeen (mutaties huurverhoging per element) * contractregels die per opgegeven huurverhogingsdatum actief zijn * voor het kale-huur-element kan op basis van de opgegeven liberale grenswaarde een afwijkend huurverhogingspercentage worden ingevoerd. Inclusief toevoeging "pp-element 031 + 281".<br><br>DRAAIBOEK: stap "Rapport vullen tabellen"<br>11-03-2020 GEKOPPELD AAN PRODUCTIE</div></td></tr></tbody></table>

### 3\. Afwijkingen op eenheden vullen in hulptabellen (rapport in rapportageportal + bijbehorende stored procedure in dwh-database)

<table class="ms-listviewtable" id="onetidDoclibViewTbl0" dir="none" border="0" cellspacing="0" cellpadding="1" summary="RapportenBoek" o:webquerysourcehref="&amp;XMLDATA=1&amp;RowLimit=0&amp;View=%7B082E8BDB%2D7144%2D41B0%2D8CD5%2DCE4F09CB41B0%7D" xmlns:o="urn:schemas-microsoft-com:office:office" handledeleteinit="true"><tbody id="tbod1-6__" isloaded="true" selectablerows="8"><tr class=" ms-itmHoverEnabled ms-itmhover s4-itm-selected" id="1,1541,0" iid="1,1541,0" setedgeborder="true"><td height="100%" class="ms-cellstyle ms-vb-title" iscallout="TRUE" isecb="TRUE"><div class="ms-vb  itx" id="1541" app="" ctxname="ctx1"><a class="ms-listlink ms-draggable" aria-label="Wijzigen Huurverhoging voor eenheden met servicecluster - PRODUCTIE, rdl Bestand" href="http://s-dwh2012-sp/Rapportenboek/Wijzigen%20Huurverhoging%20voor%20eenheden%20met%20servicecluster%20-%20PRODUCTIE.rdl" dragid="13">Wijzigen Huurverhoging voor eenheden met servicecluster - PRODUCTIE</a></div></td><td class="ms-list-itemLink-td ms-cellstyle"><div class="ms-list-itemLink  "><a title="Menu Openen" class="ms-lstItmLinkAnchor ms-ellipsis-a" id="ctx1_1541_calloutLaunchPoint" href="http://s-dwh2012-sp/Level%203%20Financin/Directie%20Financien.aspx#" ms-jsgrid-click-passthrough="true"><img class="ms-ellipsis-icon" alt="Wijzigen Huurverhoging voor eenheden met servicecluster - PRODUCTIE.rdl - menu Openen" src="http://s-dwh2012-sp/_catalogs/theme/Themed/7FA80357/spcommon-B35BB0A9.themedpng?ctag=2"></a></div></td><td class="ms-cellstyle ms-vb2 ms-vb-lastCell"><div class="ms-rtestate-field" dir="">Dit rapport kan gedraaid worden aan het eind van de voorbereiding van de huurverhoging. De tabel HuurverhogingEenheid wordt als volgt daarbij gewijzigd voor eenheden die een element hebben en gekoppeld zijn aan een servicecluster waarvoor aparte bedragen zijn opgevoerd. Dat wil zeggen waarvoor dus regels zijn opgevoerd in de tabel HuurverhogingServiceCluster.<br><br>DRAAIBOEK: stap "Rapport vullen tabellen"<br>11-03-2020 GEKOPPELD AAN PRODUCTIE</div></td></tr></tbody></table>

# 1100
## BIJLAGE 2: welke tabellen worden gekopieerd en bewaard

### <span style="font-size: 14px;">Data overhalen van testomgeving: empire_staedion_logic.dbo.sp_load_specieke_tabellen_hvh</span>

<span style="font-size: 14px;">Test (Azure): kan langer dan 1h:40 duren</span>

<span style="font-size: 14px;">Gaat om volgende tabellen:</span>

- 2021-04-03 08:55:15: VULLEN empire\_Staedion\_data.hvh.\[Staedion$Target Rent \_ Max\_ Reas\_ Rent\] -  records: 55
- 2021-04-03 08:55:16: VULLEN empire\_Staedion\_data.hvh.\[Staedion$CR-gegevens\] -  records: 25
- 2021-04-03 08:55:17: VULLEN empire\_Staedion\_data.hvh.\[Staedion$VAT Posting Setup\] -  records: 44
- 2021-04-03 08:55:23: VULLEN empire\_Staedion\_data.hvh.\[Staedion$Maximale huurprijzen\] -  records: 14,838
- 2021-04-03 08:56:10: VULLEN empire\_Staedion\_data.hvh.\[staedion$Oge\] -  records: 68,564
- 2021-04-03 09:03:43: VULLEN empire\_Staedion\_data.hvh.\[Staedion$Property Valuation\] -  records: 541,681
- 2021-04-03 09:03:44: VULLEN empire\_Staedion\_data.hvh.\[Staedion$Prop\_ Valuation (Shared Acc\_)\] -  records: 4,382
- 2021-04-03 09:08:15: VULLEN empire\_Staedion\_data.hvh.\[staedion$OGE Rent Increase\] -  records: 249,621
- 2021-04-03 09:08:43: VULLEN empire\_Staedion\_data.hvh.\[Staedion$Rent Increase Element\] -  records: 214,995
- 2021-04-03 09:08:43: VULLEN empire\_Staedion\_data.hvh.\[parameter state aid\] -  records: 8
- 2021-04-03 09:08:44: VULLEN empire\_Staedion\_data.hvh.\[Staedion$Capping Limit per Target Grp\_\] -  records: 110
- 2021-04-03 09:09:33: VULLEN empire\_Staedion\_data.hvh.\[staedion$OGE Rent Sum\] -  records: 239,634
- 2021-04-03 09:09:36: VULLEN empire\_Staedion\_data.hvh.\[staedion$OGE Rent Sum Exception\] -  records: 48,901
- 2021-04-03 09:17:14: VULLEN empire\_Staedion\_data.hvh.\[staedion$Contract\] (\[A-schiedam-sql\].emphvl2021.) -  records: 88,022
- 2021-04-03 10:34:46: VULLEN empire\_Staedion\_data.hvh.\[staedion$Element\] -  records: 1,044,090

\[empire\_dwh\].\[dbo\].\[tmv\_hvh\_simulatie\_testomgeving\] 

BRON: empire\_staedion\_data.hvh.\[Staedion$Oge Rent Increase\]  + empire\_staedion\_Data.hvh.\[Staedion$OGE\]  + empire\_data.dbo.\[Staedion$Technical\_Unit\_Type\]  + empire\_staedion\_Data.hvh.\[Staedion$Contract\] 

\[empire\_dwh\].\[dbo\].tmv\_hvh\_simulatie\_productieomgeving 

BRON: empire\_data.dbo.Staedion$Oge\_Rent\_Increase as HVH + empire\_data.dbo.\[Staedion$OGE\] + empire\_data.dbo.\[Staedion$Technical\_Unit\_Type\]  + empire\_data.dbo.\[Staedion$Contract\]

# 1200
# BIJLAGE 3: welke database-objecten zijn in gebruik

EXEC \[empire\_staedion\_data\].\[dbo\].\[dsp\_info\_object\_en\_velden\] 'empire\_staedion\_logic', 'dbo', 'sp\_load\_datamart\_hvh\_simulatie'